In [1]:
pip install pygame


Note: you may need to restart the kernel to use updated packages.


In [1]:
import itertools
import random
import copy

class Minesweeper():
    """
    Minesweeper game representation
    """

    def __init__(self, height=8, width=8, mines=8):

        # Set initial width, height, and number of mines
        self.height = height
        self.width = width
        self.mines = set()

        # Initialize an empty field with no mines
        self.board = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                row.append(False)
            self.board.append(row)

        # Add mines randomly
        while len(self.mines) != mines:
            i = random.randrange(height)
            j = random.randrange(width)
            if not self.board[i][j]:
                self.mines.add((i, j))
                self.board[i][j] = True

        # At first, player has found no mines
        self.mines_found = set()

    def print(self):
        """
        Prints a text-based representation
        of where mines are located.
        """
        for i in range(self.height):
            print("--" * self.width + "-")
            for j in range(self.width):
                if self.board[i][j]:
                    print("|X", end="")
                else:
                    print("| ", end="")
            print("|")
        print("--" * self.width + "-")

    def is_mine(self, cell):
        i, j = cell
        return self.board[i][j]

    def nearby_mines(self, cell):
        """
        Returns the number of mines that are
        within one row and column of a given cell,
        not including the cell itself.
        """

        # Keep count of nearby mines
        count = 0

        # Loop over all cells within one row and column
        for i in range(cell[0] - 1, cell[0] + 2):
            for j in range(cell[1] - 1, cell[1] + 2):

                # Ignore the cell itself
                if (i, j) == cell:
                    continue

                # Update count if cell in bounds and is mine
                if 0 <= i < self.height and 0 <= j < self.width:
                    if self.board[i][j]:
                        count += 1

        return count

    def won(self):
        """
        Checks if all mines have been flagged.
        """
        return self.mines_found == self.mines

In [2]:
class Sentence():
    """
    Logical statement about a Minesweeper game
    A sentence consists of a set of board cells,
    and a count of the number of those cells which are mines.
    """

    def __init__(self, cells, count):
        self.cells = set(cells)
        self.count = count

    def __eq__(self, other):
        return self.cells == other.cells and self.count == other.count

    def __str__(self):
        return f"{self.cells} = {self.count}"

    def known_mines(self):
        """
        Returns the set of all cells in self.cells known to be mines.
        """
        # if the set contains all mines and is not an empty set
        if len(self.cells) == self.count and len(self.cells) != 0:
            return self.cells
        print(f"Sentence known_mines() called.")
        return set()

    def known_safes(self):
        """
        Returns the set of all cells in self.cells known to be safe.
        """
        # if the set is not empty and the count of its mines is zero
        if len(self.cells) != 0 and self.count == 0:
            return self.cells
        print(f"Sentence known_safes() called.")
        return set()

    def mark_mine(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be a mine.
        """
        # if:
        first = cell in self.cells # the cell is in the sentence
        second = len(self.cells) >= 1 # the sentence isn't empty
        third = self.count >= 1 # the sentence has a mine in it
        
        if first and second and third:
            # update the sentence so that cell is no longer in the sentence
            self.cells.remove(cell)
            # and decrement the count appropriately (since removing a mine decreases the count)
            self.count -= 1
        print(f"Sentence mark_mine() called with cell {cell}")    

    def mark_safe(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be safe.
        """
        # if:
        first = cell in self.cells # the cell is in the sentence
        second = len(self.cells) >= 1 # the sentence isn't empty
        third = len(self.cells) != self.count # the sentence doesn't consist of only mines
        
        if first and second and third:
            # update the sentence so that cell is no longer in the sentence
            # there is no need to decrement 
            self.cells.remove(cell)
        print(f"Sentence mark_safe() called with cell {cell}")

In [3]:
class MinesweeperAI():
    """
    Minesweeper game player
    """

    def __init__(self, height=8, width=8):

        # Set initial height and width
        self.height = height
        self.width = width

        # Keep track of which cells have been clicked on
        self.moves_made = set()

        # Keep track of cells known to be safe or mines
        self.mines = set()
        self.safes = set()

        # List of sentences about the game known to be true
        self.knowledge = []
        
        self.all_possible_moves = set()
        for i in range(1,height+1):
            for j in range(1,width+1):
                self.all_possible_moves.add((i,j))

    def mark_mine(self, cell):
        """
        Marks a cell as a mine, and updates all knowledge
        to mark that cell as a mine as well.
        """
        self.mines.add(cell)
        for sentence in self.knowledge:
            sentence.mark_mine(cell)
        print(f"MinesweeperAI mark_mine() called with cell {cell}")

    def mark_safe(self, cell):
        """
        Marks a cell as safe, and updates all knowledge
        to mark that cell as safe as well.
        """
        self.safes.add(cell)
        for sentence in self.knowledge:
            sentence.mark_safe(cell)
        print(f"MinesweeperAI mark_safe() called with cell {cell}")

    def add_knowledge(self, cell, count):
        """
        Called when the Minesweeper board tells us, for a given
        safe cell, how many neighboring cells have mines in them.

        This function should:
            1) mark the cell as a move that has been made
            2) mark the cell as safe
            3) add a new sentence to the AI's knowledge base
               based on the value of `cell` and `count`
            4) mark any additional cells as safe or as mines
               if it can be concluded based on the AI's knowledge base
            5) add any new sentences to the AI's knowledge base
               if they can be inferred from existing knowledge
        """
#       1) mark the cell as a move that has been made
        self.moves_made.add(cell)
        print(f"MinesweeperAI add_knowledge() self.moves_made(): {self.moves_made}")
#       2) mark the cell as safe
        self.mark_safe(cell)
        print(f"MinesweeperAI add_knowledge() self.mark_safe() called.")
#       3) add a new sentence to the AI's knowledge base
#                based on the value of `cell` and `count`
        
        # find all neighbouring cells
        cell_neighbours = set()
        # loop through all cell possibilities around the target cell
        for i in range(cell[0] - 1, cell[0] + 2):
            for j in range(cell[1] - 1, cell[1] + 2):
                # Ignore the cell itself
                if (i, j) == cell:
                    continue
                # take the neighbours within the bounds
                if 0 <= i < self.height and 0 <= j < self.width:
                    cell_neighbours.add((i,j))
        print(f"MinesweeperAI add_knowledge() cell_neighbours: {cell_neighbours}")
        # add a new sentence to knowledge base containing the cells and how many mines there are
        new_info = Sentence(cell_neighbours,count)
        print(f"MinesweeperAI add_knowledge() new_info: {new_info}")
        self.knowledge.append(new_info)        

#        4) mark any additional cells as safe or as mines
#          if it can be concluded based on the AI's knowledge base
        
        # make a copy of the knowledge base
        know2 = copy.copy(self.knowledge)
        # go through each sentence in the copy
        for sentence in know2:
            # update sentences as necessary
            # if it's safe:
            if len(sentence.cells) == 1 and sentence.count == 0:
                self.mark_safe(sentence.cells)
            # if it's a mine:
            if len(sentence.cells) == 1 and sentence.count >= 1:
                self.mark_mine(sentence.cells)
        
        self.knowledge = know2
        
#       5) add any new sentences to the AI's knowledge base
#          if they can be inferred from existing knowledge

#       5a) By extension, any time we have a sentence whose count is 0, we know that 
#           all of that sentence’s cells must be safe.
        
        # go through each sentence in knowledge base
        for sentence in know2:
            # if the sentence has a count of zero
            if sentence.count == 0:
                # all the sentence's cells are safe
                for cell in sentence.cells:
                    self.mark_safe(cell)

#       5b) More generally, any time the number of cells is equal to the count, we know that 
#           all of that sentence’s cells must be mines
        
        # go through each sentence in knowledge base
        for sentence in self.knowledge:
            # if the number of cells is equal to the count
            if len(sentence.cells) == sentence.count:
                # all of the sentence's cells are mines
                for cell in copy.deepcopy(sentence.cells):
                    self.mark_mine(cell)
        
#       5c) More generally, any time we have two sentences set1 = count1 and set2 = count2 
#           where set1 is a subset of set2, then we can construct the new sentence set2 - set1 = count2 - count1
        
        # go through each sentence in knowledge base
        for sentence1 in self.knowledge:
            # compare it against each other sentence in knowledge base
            for sentence2 in self.knowledge:
                if sentence1 != sentence2:
                    # if set1 is a subset of set2
                    if sentence1.cells.issubset(sentence2.cells):
                        # construct the new sentence set2 - set1 = count2 - count1
                        self.knowledge.append(Sentence(
                            cells = sentence2.cells.difference(sentence1.cells), 
                            count = sentence2.count - sentence1.count))

    def make_safe_move(self):
        """
        Returns a safe cell to choose on the Minesweeper board.
        The move must be known to be safe, and not already a move
        that has been made.

        This function may use the knowledge in self.mines, self.safes
        and self.moves_made, but should not modify any of those values.
        """
        # if there are no safe moves, return nothing
        if len(self.safes) == 0:
            return None
        # otherwise, find the set of safe moves you can make that you haven't already
        diff = self.safes.difference(self.moves_made)

        # choose an item from the set and return it (convert to tuple to be able to use random.choice)
        print(f"Make_safe_move: diff -  {diff}")
        return random.choice(tuple(diff))


    def make_random_move(self):
        """
        Returns a move to make on the Minesweeper board.
        Should choose randomly among cells that:
            1) have not already been chosen, and
            2) are not known to be mines
        """
        # if there are no possible moves to make
        if len(all_moves) == len(self.moves_made):
            return None
        
        # all moves possible were calculated on init based on size of board (height, width)
        all_moves = self.all_possible_moves

        # illegal moves are those that have already been made or are mines
        illegal_moves = self.moves_made.union(self.mines)

        # allowed moves are those on the board that aren't "illegal"
        allowed_moves = all_moves.difference(illegal_moves)

        # randomly pick one move to make from the set
        choice = random.choice(tuple(allowed_moves))
        print(f"Make_random_move: choice - {choice}")
        return choice


In [4]:
import pygame
import sys
import time

# from minesweeper_stw import Minesweeper, MinesweeperAI

# HEIGHT = 8
# WIDTH = 8
# MINES = 8

HEIGHT = 3
WIDTH = 3
MINES = 2

# Colors
BLACK = (0, 0, 0)
GRAY = (180, 180, 180)
WHITE = (255, 255, 255)

# Create game
pygame.init()
size = width, height = 600, 400
screen = pygame.display.set_mode(size)

# Fonts
OPEN_SANS = "assets/fonts/OpenSans-Regular.ttf"
smallFont = pygame.font.Font(OPEN_SANS, 20)
mediumFont = pygame.font.Font(OPEN_SANS, 28)
largeFont = pygame.font.Font(OPEN_SANS, 40)

# Compute board size
BOARD_PADDING = 20
board_width = ((2 / 3) * width) - (BOARD_PADDING * 2)
board_height = height - (BOARD_PADDING * 2)
cell_size = int(min(board_width / WIDTH, board_height / HEIGHT))
board_origin = (BOARD_PADDING, BOARD_PADDING)

# Add images
flag = pygame.image.load("assets/images/flag.png")
flag = pygame.transform.scale(flag, (cell_size, cell_size))
mine = pygame.image.load("assets/images/mine.png")
mine = pygame.transform.scale(mine, (cell_size, cell_size))

# Create game and AI agent
game = Minesweeper(height=HEIGHT, width=WIDTH, mines=MINES)
ai = MinesweeperAI(height=HEIGHT, width=WIDTH)

# Keep track of revealed cells, flagged cells, and if a mine was hit
revealed = set()
flags = set()
lost = False

# Show instructions initially
instructions = True

while True:

    # Check if game quit
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()

    screen.fill(BLACK)

    # Show game instructions
    if instructions:

        # Title
        title = largeFont.render("Play Minesweeper", True, WHITE)
        titleRect = title.get_rect()
        titleRect.center = ((width / 2), 50)
        screen.blit(title, titleRect)

        # Rules
        rules = [
            "Click a cell to reveal it.",
            "Right-click a cell to mark it as a mine.",
            "Mark all mines successfully to win!"
        ]
        for i, rule in enumerate(rules):
            line = smallFont.render(rule, True, WHITE)
            lineRect = line.get_rect()
            lineRect.center = ((width / 2), 150 + 30 * i)
            screen.blit(line, lineRect)

        # Play game button
        buttonRect = pygame.Rect((width / 4), (3 / 4) * height, width / 2, 50)
        buttonText = mediumFont.render("Play Game", True, BLACK)
        buttonTextRect = buttonText.get_rect()
        buttonTextRect.center = buttonRect.center
        pygame.draw.rect(screen, WHITE, buttonRect)
        screen.blit(buttonText, buttonTextRect)

        # Check if play button clicked
        click, _, _ = pygame.mouse.get_pressed()
        if click == 1:
            mouse = pygame.mouse.get_pos()
            if buttonRect.collidepoint(mouse):
                instructions = False
                time.sleep(0.3)

        pygame.display.flip()
        continue

    # Draw board
    cells = []
    for i in range(HEIGHT):
        row = []
        for j in range(WIDTH):

            # Draw rectangle for cell
            rect = pygame.Rect(
                board_origin[0] + j * cell_size,
                board_origin[1] + i * cell_size,
                cell_size, cell_size
            )
            pygame.draw.rect(screen, GRAY, rect)
            pygame.draw.rect(screen, WHITE, rect, 3)

            # Add a mine, flag, or number if needed
            if game.is_mine((i, j)) and lost:
                screen.blit(mine, rect)
            elif (i, j) in flags:
                screen.blit(flag, rect)
            elif (i, j) in revealed:
                neighbors = smallFont.render(
                    str(game.nearby_mines((i, j))),
                    True, BLACK
                )
                neighborsTextRect = neighbors.get_rect()
                neighborsTextRect.center = rect.center
                screen.blit(neighbors, neighborsTextRect)

            row.append(rect)
        cells.append(row)

    # AI Move button
    aiButton = pygame.Rect(
        (2 / 3) * width + BOARD_PADDING, (1 / 3) * height - 50,
        (width / 3) - BOARD_PADDING * 2, 50
    )
    buttonText = mediumFont.render("AI Move", True, BLACK)
    buttonRect = buttonText.get_rect()
    buttonRect.center = aiButton.center
    pygame.draw.rect(screen, WHITE, aiButton)
    screen.blit(buttonText, buttonRect)

    # Reset button
    resetButton = pygame.Rect(
        (2 / 3) * width + BOARD_PADDING, (1 / 3) * height + 20,
        (width / 3) - BOARD_PADDING * 2, 50
    )
    buttonText = mediumFont.render("Reset", True, BLACK)
    buttonRect = buttonText.get_rect()
    buttonRect.center = resetButton.center
    pygame.draw.rect(screen, WHITE, resetButton)
    screen.blit(buttonText, buttonRect)

    # Display text
    text = "Lost" if lost else "Won" if game.mines == flags else ""
    text = mediumFont.render(text, True, WHITE)
    textRect = text.get_rect()
    textRect.center = ((5 / 6) * width, (2 / 3) * height)
    screen.blit(text, textRect)

    move = None

    left, _, right = pygame.mouse.get_pressed()

    # Check for a right-click to toggle flagging
    if right == 1 and not lost:
        mouse = pygame.mouse.get_pos()
        for i in range(HEIGHT):
            for j in range(WIDTH):
                if cells[i][j].collidepoint(mouse) and (i, j) not in revealed:
                    if (i, j) in flags:
                        flags.remove((i, j))
                    else:
                        flags.add((i, j))
                    time.sleep(0.2)

    elif left == 1:
        mouse = pygame.mouse.get_pos()

        # If AI button clicked, make an AI move
        if aiButton.collidepoint(mouse) and not lost:
            move = ai.make_safe_move()
            if move is None:
                move = ai.make_random_move()
                if move is None:
                    flags = ai.mines.copy()
                    print("No moves left to make.")
                else:
                    print("No known safe moves, AI making random move.")
            else:
                print("AI making safe move.")
            time.sleep(0.2)

        # Reset game state
        elif resetButton.collidepoint(mouse):
            game = Minesweeper(height=HEIGHT, width=WIDTH, mines=MINES)
            ai = MinesweeperAI(height=HEIGHT, width=WIDTH)
            revealed = set()
            flags = set()
            lost = False
            continue

        # User-made move
        elif not lost:
            for i in range(HEIGHT):
                for j in range(WIDTH):
                    if (cells[i][j].collidepoint(mouse)
                            and (i, j) not in flags
                            and (i, j) not in revealed):
                        move = (i, j)

    # Make move and update AI knowledge
    if move:
        if game.is_mine(move):
            lost = True
        else:
            nearby = game.nearby_mines(move)
            revealed.add(move)
            ai.add_knowledge(move, nearby)

    pygame.display.flip()


pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
MinesweeperAI add_knowledge() self.moves_made(): {(0, 1)}
MinesweeperAI mark_safe() called with cell (0, 1)
MinesweeperAI add_knowledge() self.mark_safe() called.
MinesweeperAI add_knowledge() cell_neighbours: {(1, 2), (0, 0), (1, 1), (0, 2), (1, 0)}
MinesweeperAI add_knowledge() new_info: {(0, 2), (1, 2), (0, 0), (1, 0), (1, 1)} = 2


TypeError: 'module' object is not callable